In [1]:
import torch
import torchvision
import torch.nn as nn
from torch.nn import functional as F
import torchvision.transforms as transforms
import torch.optim as optim
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

class Dataset(torch.utils.data.Dataset):
    def __init__(self, transforms=transform):
        self.data = []
        rootdir = 'train'
        self.transforms = transform
        for subdir, dirs, files in os.walk('train/NORMAL'):
            for file in files:
                filepath = subdir + os.sep + file
                self.data.append([self.transforms(Image.open(filepath).convert("RGB")), 0])
        for subdir, dirs, files in os.walk('train/PNEUMONIA'):
            for file in files:
                filepath = subdir + os.sep + file
                self.data.append([self.transforms(Image.open(filepath).convert("RGB")), 1])
        
    def __getitem__(self, idx):
        return self.data[idx]
    
    def __len__(self):
        return len(self.data)
    
traindata = Dataset()

In [3]:
class Valdata(torch.utils.data.Dataset):
    def __init__(self, transforms=transform):
        self.data = []
        rootdir = 'val'
        self.transforms = transform
        for subdir, dirs, files in os.walk('val/NORMAL'):
            for file in files:
                filepath = subdir + os.sep + file
                self.data.append([self.transforms(Image.open(filepath).convert("RGB")), 0])
        for subdir, dirs, files in os.walk('val/PNEUMONIA'):
            for file in files:
                filepath = subdir + os.sep + file
                self.data.append([self.transforms(Image.open(filepath).convert("RGB")), 1])
        
    def __getitem__(self, idx):
        return self.data[idx]
    
    def __len__(self):
        return len(self.data)
    
valdata = Valdata()

In [4]:
class Testdata(torch.utils.data.Dataset):
    def __init__(self, transforms=transform):
        self.data = []
        rootdir = 'test'
        self.transforms = transform
        for subdir, dirs, files in os.walk('test/NORMAL'):
            for file in files:
                filepath = subdir + os.sep + file
                self.data.append([self.transforms(Image.open(filepath).convert("RGB")), 0])
        for subdir, dirs, files in os.walk('test/PNEUMONIA'):
            for file in files:
                filepath = subdir + os.sep + file
                self.data.append([self.transforms(Image.open(filepath).convert("RGB")), 1])
        
    def __getitem__(self, idx):
        return self.data[idx]
    
    def __len__(self):
        return len(self.data)
    
testdata = Testdata()

In [5]:
trainloader = torch.utils.data.DataLoader(traindata, batch_size=32, shuffle=True)
valloader = torch.utils.data.DataLoader(valdata, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(testdata, batch_size=1, shuffle=True)

In [7]:
model = torchvision.models.resnet152(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1000, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = F.relu(self.fc3(x))
        return x

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /home/gpu-station/.cache/torch/checkpoints/resnet152-b121ed2d.pth


In [8]:
mynet = Net()
net = nn.Sequential(model, mynet)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [15]:
optimizer = optim.Adam(net.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
criterion = nn.BCEWithLogitsLoss()

In [17]:
%%time
net.to(device)
for epoch in range(20):
    running_loss = 0
    net.train()
    for images, labels in trainloader:
        images = images.to(device); labels = labels.to(device)
        outputs = net(images); labels = labels.float()
        outputs = outputs.reshape(labels.shape)
        optimizer.zero_grad()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
    scheduler.step()
    net.eval()
    val_loss = 0
    for images, labels in valloader:
        images = images.to(device); labels = labels.to(device)
        outputs = net(images); labels = labels.float()
        outputs = outputs.reshape(labels.shape)
        loss = criterion(outputs, labels)
        val_loss+=loss.item()
    print("Epoch {} elapsed -----> loss:{:5f}, val_loss:{:5f}".format(epoch, running_loss/len(trainloader), val_loss/len(valloader)))

Epoch 0 elapsed -----> loss:0.331467, val_loss:1.899046
Epoch 1 elapsed -----> loss:0.320871, val_loss:1.200640
Epoch 2 elapsed -----> loss:0.315729, val_loss:1.092165
Epoch 3 elapsed -----> loss:0.311224, val_loss:1.381870
Epoch 4 elapsed -----> loss:0.305054, val_loss:1.805350
Epoch 5 elapsed -----> loss:0.313120, val_loss:2.165716
Epoch 6 elapsed -----> loss:0.312695, val_loss:1.942808
Epoch 7 elapsed -----> loss:0.303045, val_loss:1.462704
Epoch 8 elapsed -----> loss:0.304188, val_loss:1.852155
Epoch 9 elapsed -----> loss:0.290924, val_loss:1.610152
Epoch 10 elapsed -----> loss:0.289441, val_loss:1.434570
Epoch 11 elapsed -----> loss:0.286071, val_loss:1.532824
Epoch 12 elapsed -----> loss:0.291996, val_loss:1.297253
Epoch 13 elapsed -----> loss:0.283306, val_loss:1.535838
Epoch 14 elapsed -----> loss:0.284575, val_loss:1.428553
Epoch 15 elapsed -----> loss:0.288662, val_loss:1.492237
Epoch 16 elapsed -----> loss:0.286774, val_loss:1.465596
Epoch 17 elapsed -----> loss:0.279363, va

In [29]:
%%time
y_hat = []
y = []
loss_on_test = 0
for images, labels in testloader:
    images = images.to(device); labels = labels.to(device)
    outputs = net(images); labels = labels.float()
    outputs = outputs.reshape(labels.shape)
    loss = criterion(outputs, labels)
    loss_on_test += loss.item()
    y.append(labels.item())
    y_hat.append((nn.Sigmoid()(outputs)).item())
assert len(y_hat)==len(y)

CPU times: user 8.01 s, sys: 15.8 ms, total: 8.03 s
Wall time: 8.02 s


In [40]:
y_hat = np.array(y_hat); y = np.array(y)
y_hat[y_hat<=0.5] = 0; y_hat[y_hat>0.5]=1
accuracy_score = np.mean(np.absolute(y_hat - y))

In [43]:
print(accuracy_score)
print(loss_on_test/len(testloader))

0.14903846153846154
0.5305528369661624
